<a href="https://colab.research.google.com/github/jchen8000/MachineLearning/blob/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/context_conditional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context-Conditional GAN

In [0]:
from keras.models import model_from_json
import numpy as np
import matplotlib.pyplot as plt

In [0]:
!wget https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator.json
!wget https://raw.githubusercontent.com/jchen8000/MachineLearning/master/10%20Generative%20Adversarial%20Network/Context-Conditional%20GAN/saved_model/ccgan_generator_weights.hdf5


In [0]:
def load_model(model_name):
    model_path = "%s.json" % model_name
    weights_path = "%s_weights.hdf5" % model_name
    
    
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)
    
    return loaded_model
  

generator = load_model("ccgan_generator")
#discriminator = load_model("ccgan_discriminator")


In [0]:

def sample_images(self, epoch, imgs):
    r, c = 3, 6

    masked_imgs = self.mask_randomly(imgs)
    gen_imgs = self.generator.predict(masked_imgs)

    imgs = (imgs + 1.0) * 0.5
    masked_imgs = (masked_imgs + 1.0) * 0.5
    gen_imgs = (gen_imgs + 1.0) * 0.5

    gen_imgs = np.where(gen_imgs < 0, 0, gen_imgs)

    fig, axs = plt.subplots(r, c)
    for i in range(c):
        axs[0,i].imshow(imgs[i, :, :, 0], cmap='gray')
        axs[0,i].axis('off')
        axs[1,i].imshow(masked_imgs[i, :, :, 0], cmap='gray')
        axs[1,i].axis('off')
        axs[2,i].imshow(gen_imgs[i, :, :, 0], cmap='gray')
        axs[2,i].axis('off')
    plt.show()